<a href="https://colab.research.google.com/github/SolutionLr/DL/blob/main/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary modules 
from google.colab import drive
drive.mount('/content/drive')
from torchvision import models
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import argparse
import torch
import torchvision
import torch.nn as nn
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data
def load_image(image_path, transform=None):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image.to(device)


transform = transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.ToTensor(),
])


# Create model
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28']
        self.vgg = models.vgg19(pretrained=True).features

    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features


model = VGGNet().to(device).eval()


# Load images
content = load_image('/content/drive/MyDrive/DL_1/Neural Style Transfer/content.jpg', transform)
style = load_image('/content/drive/MyDrive/DL_1/Neural Style Transfer/style.jpg', transform)
generated = content.clone().requires_grad_(True)

# train
optimizer = torch.optim.Adam([generated], lr=0.03)
epoch = 2000
for i in range(epoch):
    c_f = model(content)
    s_f = model(style)
    g_f = model(generated)
    j_content = 0
    j_style = 0
    for f1, f2, f3 in zip(c_f, s_f, g_f):
        _, C, H, W = list(f3.size())
        j_content += 1 / (4 * C * H * W) * torch.sum(torch.square(torch.subtract(f1, f3)))
        Gg = f3.view(C, H*W)
        Gg = torch.mm(Gg, Gg.t())
        Gs = f2.view(C, H*W)
        Gs = torch.mm(Gs, Gs.t())
        j_style += 1/(H**2 * H**2 * W **2) * torch.sum(torch.square(torch.subtract(Gg, Gs)))
    optimizer.zero_grad()
    J = j_content + 1/5 * j_style
    J.backward()
    optimizer.step()
    if i%10==0 and i!=0:
        print(f"Loss ==> {J} n_steps ==>[{i}/2000]")
        save_image(generated, 'generated.png')
